In [167]:
import pandas as pd
import xgboost as xgb
from datetime import timedelta
import numpy as np

In [168]:
bst = xgb.Booster({'nthread':4}) # init model
bst.load_model("./xgboosting.model")      # load data

In [169]:
test_df = pd.read_csv('../Data/test.csv')

In [170]:
tags = ['Atmospheric',
 'Great Soundtrack',
 'Story Rich',
 'Multiplayer',
 'RPG',
 'Open World',
 'Strategy',
 'Co-op',
 'Sci-fi',
 'Fantasy',
 'Masterpiece',
 'Simulation',
 'First-Person',
 '2D',
 'Puzzle',
 'Third Person',
 'Shooter',
 'Funny',
 'Casual',
 'Horror',
 'Sandbox',
 'Difficult',
 'Survival',
 'Exploration',
 'Female Protagonist',
 'Early Access',
 'Comedy',
 'FPS',
 'Gore',
 'Point & Click',
 'Online Co-Op',
 'Choices Matter',
 'Classic',
 'Space',
 'VR',
 'Turn-Based',
 'Platformer',
 'Violent',
 'Dark',
 'Moddable',
 'Hack and Slash',
 'Local Co-Op',
 'Mystery',
 'Stealth',
 'Nudity',
 'Retro',
 'Psychological Horror',
 'Action RPG',
 'Building',
 'Third-Person Shooter',
 'Replay Value',
 'Isometric',
 'Pixel Graphics',
 'Mature',
 'Walking Simulator',
 'Short',
 'Character Customization',
 'Tactical',
 'Free to Play',
 'Post-apocalyptic',
 'Massively Multiplayer',
 'Crafting',
 'Controller',
 'Rogue-like',
 'RTS',
 'Historical']
len(cats)

24

In [171]:
genres = ['Adventure',
 'Casual',
 'Indie',
 'RPG',
 'Action',
 'Strategy',
 'Simulation',
 'Racing',
 'Sports',
 'Massively Multiplayer',
 'Sexual Content',
 'Violent',
 'Free to Play',
 'Early Access',
 'Audio Production',
 'Gore',
 'Design & Illustration',
 'Nudity',
 'Animation & Modeling',
 'Utilities']
len(genres)

20

In [172]:
cats = ['Multi-player',
 'Co-op',
 'Partial Controller Support',
 'Steam Workshop',
 'Steam Leaderboards',
 'Online Multi-Player',
 'Includes level editor',
 'Shared/Split Screen',
 'Cross-Platform Multiplayer',
 'Online Co-op',
 'Captions available',
 'Stats',
 'Local Co-op',
 'Remote Play on Tablet',
 'Remote Play on Phone',
 'In-App Purchases',
 'Local Multi-Player',
 'Valve Anti-Cheat enabled',
 'Remote Play on TV',
 'VR Support',
 'Commentary available',
 'MMO',
 'SteamVR Collectibles',
 'Includes Source SDK']
len(tags)

66

In [173]:
def predict_data_prpare(data):
    X = []
    for item in data.iterrows():
        row = item[1]
        tmp = []
        # genres
        row_g = row['genres'].split(',')
        for g in genres:
            if g in row_g:
                tmp.append(1)
            else:
                tmp.append(0)
        # cats
        row_c = row['categories'].split(',')
        for c in cats:
            if c in row_c:
                tmp.append(1)
            else:
                tmp.append(0)
        # tags
        row_t = row['tags'].split(',')
        for t in tags:
            if t in row_t:
                tmp.append(1)
            else:
                tmp.append(0)
        # is_free
        if row['is_free']:
            tmp.append(1)
        else:
            tmp.append(0)
        # price
        tmp.append(row['price'])
        # timegap
        if not np.isnan(row['time_gap']):
            tmp.append(row['time_gap'])
        else:
            tmp.append(0.5)
        # average_review
        if not np.isnan(row['average_review']):
            tmp.append(row['average_review'])
        else:
            tmp.append(0.5)
        
        X.append(tmp)
    
    return X

In [174]:
test_df['average_review'] = test_df['total_positive_reviews'] / (test_df['total_positive_reviews'] + test_df['total_negative_reviews'])

In [175]:
test_df['purchase_date'] = pd.to_datetime(test_df['purchase_date'], infer_datetime_format=True)
test_df['release_date'] = pd.to_datetime(test_df['release_date'], infer_datetime_format=True)

In [176]:
test_df['time_gap'] = test_df['purchase_date'] - test_df['release_date']

In [177]:
test_df['time_gap'] = (test_df['time_gap'] - timedelta(-481)) / (timedelta(4320) - timedelta(-481))

In [178]:
test_df['price'] = test_df['price'] / 15999900.0

In [179]:
# test_df

In [180]:
X = predict_data_prpare(test_df)

In [181]:
test_x = xgb.DMatrix(X)

In [182]:
ans = bst.predict(test_x)

In [183]:
ans *= 113.8

In [184]:
ans

array([2.13674389e-09, 1.50542210e-08, 2.75389789e-06, 2.78837608e-20,
       3.23203822e-05, 2.57898658e-03, 9.72016245e-09, 3.17779445e-16,
       8.71901943e-12, 1.70453684e-04, 5.00316582e-06, 3.80072967e-14,
       1.51936927e-12, 1.48062361e-06, 8.84397309e-07, 3.22365165e-02,
       7.72679895e-02, 4.79312788e-04, 3.31774885e-07, 3.83327744e-04,
       4.04505504e-13, 6.18646108e-02, 2.36978622e-07, 6.91878277e-10,
       2.41139252e-03, 1.16573131e-06, 2.25723953e-12, 3.55003518e-03,
       5.87243594e-05, 8.95740639e-04, 1.62219185e-05, 7.51329895e+02,
       8.85380505e-05, 1.42486804e-11, 1.26104176e+00, 4.88465620e-18,
       5.90504169e-05, 1.37968123e-01, 5.68098173e-08, 6.25089986e-13,
       1.32111187e+01, 1.86445481e-09, 4.22764234e-02, 1.21895425e-07,
       1.47519866e-04, 9.19172560e-09, 1.36806350e-02, 6.29970451e-11,
       2.11375272e-16, 1.01494521e-01, 3.67916538e-03, 2.78228107e-10,
       4.97340155e-11, 4.12305967e-09, 9.49744208e-05, 4.32672481e-10,
      

In [185]:
for i in range(len(ans)):
    ans[i] = round(ans[i], 2)

In [186]:
ans

array([0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
       0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
       0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
       3.00000e-02, 8.00000e-02, 0.00000e+00, 0.00000e+00, 0.00000e+00,
       0.00000e+00, 6.00000e-02, 0.00000e+00, 0.00000e+00, 0.00000e+00,
       0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
       0.00000e+00, 7.51330e+02, 0.00000e+00, 0.00000e+00, 1.26000e+00,
       0.00000e+00, 0.00000e+00, 1.40000e-01, 0.00000e+00, 0.00000e+00,
       1.32100e+01, 0.00000e+00, 4.00000e-02, 0.00000e+00, 0.00000e+00,
       0.00000e+00, 1.00000e-02, 0.00000e+00, 0.00000e+00, 1.00000e-01,
       0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
       0.00000e+00, 8.00000e-02, 9.00000e-02, 0.00000e+00, 1.00000e-02,
       0.00000e+00, 0.00000e+00, 0.00000e+00, 2.76998e+03, 0.00000e+00,
       0.00000e+00, 1.50000e-01, 0.00000e+00, 6.00000e-02, 1.000

In [188]:
ans[63]

2769.98

In [262]:
pd.DataFrame({'id': [i for i in range(90)], 'playtime_forever': ans}, columns=['id', 'playtime_forever']).to_csv('result1.csv')

In [148]:
t = np.isnan(X)

In [149]:
for i in range(len(t)):
    for k in range(len(t[0])):
        if t[i][k]:
            print(k)

In [192]:
len(X)

90

In [45]:
# all
# array([0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
#        0.000000e+00, 2.000000e-02, 0.000000e+00, 0.000000e+00,
#        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
#        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
#        1.000000e-02, 0.000000e+00, 0.000000e+00, 0.000000e+00,
#        0.000000e+00, 1.000000e-02, 0.000000e+00, 0.000000e+00,
#        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
#        0.000000e+00, 0.000000e+00, 0.000000e+00, 1.442250e+03,
#        0.000000e+00, 0.000000e+00, 5.000000e-02, 0.000000e+00,
#        0.000000e+00, 4.060000e+00, 0.000000e+00, 0.000000e+00,
#        2.290000e+01, 0.000000e+00, 0.000000e+00, 0.000000e+00,
#        0.000000e+00, 0.000000e+00, 1.000000e-02, 0.000000e+00,
#        0.000000e+00, 1.000000e-02, 0.000000e+00, 0.000000e+00,
#        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
#        1.000000e-02, 7.200000e-01, 0.000000e+00, 0.000000e+00,
#        0.000000e+00, 0.000000e+00, 0.000000e+00, 3.613406e+04,
#        0.000000e+00, 0.000000e+00, 6.000000e-02, 0.000000e+00,
#        1.400000e-01, 1.700000e-01, 0.000000e+00, 0.000000e+00,
#        0.000000e+00, 0.000000e+00, 0.000000e+00, 3.000000e-02,
#        0.000000e+00, 1.120000e+00, 0.000000e+00, 0.000000e+00,
#        0.000000e+00, 1.000000e-02, 0.000000e+00, 1.600000e-01,
#        1.000000e-02, 0.000000e+00, 0.000000e+00, 0.000000e+00,
#        3.180000e+00, 2.000000e-02], dtype=float32)